In [10]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_data_dir = 'D:/Desktopshifted/MTech - PES university/Sem-3/CapstoneProject/Dataset/archive/COVID-19_Radiography_Dataset/SubData4/SubData4Output/train/'
test_data_dir = 'D:/Desktopshifted/MTech - PES university/Sem-3/CapstoneProject/Dataset/archive/COVID-19_Radiography_Dataset/SubData4/SubData4Output/test/'
val_data_dir = 'D:/Desktopshifted/MTech - PES university/Sem-3/CapstoneProject/Dataset/archive/COVID-19_Radiography_Dataset/SubData4/SubData4Output/val/'

In [3]:
input_shape = (224,224,3)
input_layer = Input(shape = (224, 224, 3))

In [13]:
img_width , img_height = [224,224]
batch_size = 16

In [11]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True)

Found 3702 images belonging to 4 classes.


In [17]:
test_set = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
       )

Found 794 images belonging to 4 classes.


In [14]:
val_set = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
       )

Found 796 images belonging to 4 classes.


In [18]:
train_y=train_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [19]:
train_set.class_indices

{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral_Pneumonia': 3}

In [12]:
import numpy as np

In [14]:
from sklearn.utils.class_weight import compute_class_weight

In [15]:
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_y),
                                        y = train_y                                                    
                                    )

In [16]:
class_weights = dict(zip(np.unique(train_y), class_weights))

In [17]:
from tensorflow.keras.models import load_model

In [18]:
VGG16_base_model = load_model('vgg16_last4layertrainble_model.h5')

In [21]:
VGG16_base_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [30]:
from keras.models import Model

In [31]:
model = Model(inputs=VGG16_base_model.input, outputs=VGG16_base_model.output)

In [32]:
lr = 0.0001

In [33]:
optimizer = 'adam'

In [34]:
from tensorflow.keras.optimizers import Adam, Adamax

In [35]:
optm = Adam(learning_rate=0.0001)

In [38]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [42]:
model.compile(optimizer=optm, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [43]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint

In [44]:
EarlyStopping = EarlyStopping(monitor='val_accuracy',
                              min_delta=.01,
                              patience=6,
                              verbose=1,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=True)

In [45]:
rlr = ReduceLROnPlateau( monitor='val_accuracy',
                            factor=0.01,
                            patience=6,
                            verbose=0,
                            mode='max',
                            min_delta=0.01)

In [46]:
model_save = ModelCheckpoint('./vgg16_last4layertrainble_model_class_imbalance_class_weights.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)

In [49]:
nb_train_samples = 3702 # number of training-samples
nb_test_samples = 794 # number of test-samples
nb_validation_samples = 796 # number of validation-samples
epochs = 10  
batch_size  = 16

In [50]:
model_history = model.fit(train_set,steps_per_epoch = nb_train_samples // batch_size,epochs = epochs,validation_data = test_set,callbacks=[EarlyStopping, model_save,rlr],class_weight=class_weights)

Epoch 1/10
231/231 [==============================] - ETA: 0s - loss: 0.3169 - accuracy: 0.8475
Epoch 1: val_loss improved from inf to 0.52209, saving model to .\vgg16_last4layertrainble_model_class_imbalance_class_weights.h5
231/231 [==============================] - 878s 4s/step - loss: 0.3169 - accuracy: 0.8475 - val_loss: 0.5221 - val_accuracy: 0.8438 - lr: 1.0000e-04
Epoch 2/10
231/231 [==============================] - ETA: 0s - loss: 0.3020 - accuracy: 0.8551
Epoch 2: val_loss did not improve from 0.52209
231/231 [==============================] - 893s 4s/step - loss: 0.3020 - accuracy: 0.8551 - val_loss: 2.0557 - val_accuracy: 0.4824 - lr: 1.0000e-04
Epoch 3/10
231/231 [==============================] - ETA: 0s - loss: 0.2691 - accuracy: 0.8717
Epoch 3: val_loss did not improve from 0.52209
231/231 [==============================] - 851s 4s/step - loss: 0.2691 - accuracy: 0.8717 - val_loss: 0.6430 - val_accuracy: 0.7683 - lr: 1.0000e-04
Epoch 4/10
231/231 [=====================

In [2]:
from tensorflow.keras.models import load_model

In [3]:
model = load_model('../vgg16_last4layertrainble_model_class_imbalance_class_weights.h5')

In [4]:
from sklearn.metrics import classification_report,confusion_matrix

In [20]:
import tqdm
val_set.reset()
X_val, y_val = next(val_set)
for i in tqdm.tqdm(range(int(val_set.n/batch_size)-1)): 
    img, label = next(val_set)
    X_val = np.append(X_val, img, axis=0 )
    y_val = np.append(y_val, label, axis=0)
print(X_val.shape, y_val.shape)

  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  2%|█▋                                                                                 | 1/48 [00:00<00:08,  5.68it/s]

<IPython.core.display.Javascript object>

...

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 47/48 [00:15<00:00,  2.22it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:15<00:00,  3.06it/s]

(784, 224, 224, 3) (784, 4)


In [21]:
score = model.evaluate(X_val, y_val, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  0.6992152333259583
Test accuracy:  0.8354591727256775


In [22]:
#predict
y_pred=model.predict(X_val)
y_pred_classes=np.argmax(y_pred,axis=1)

25/25 [==============================] - 178s 7s/step


<IPython.core.display.Javascript object>

In [23]:
y_pred_classes = tf.keras.utils.to_categorical(y_pred_classes, num_classes=4)

<IPython.core.display.Javascript object>

In [24]:
y_pred_classes

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [25]:
#get classification report
print(classification_report(y_pred_classes,y_val))

              precision    recall  f1-score   support

           0       0.91      0.72      0.81       169
           1       0.69      0.92      0.79       166
           2       0.88      0.83      0.86       402
           3       0.96      0.98      0.97        47

   micro avg       0.84      0.84      0.84       784
   macro avg       0.86      0.86      0.85       784
weighted avg       0.85      0.84      0.84       784
 samples avg       0.84      0.84      0.84       784

